In [6]:
from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

# Define pet table

In [7]:
class Pet(Base):
    __tablename__ = 'pet'
    id = Column(Integer, primary_key=True)
    name = Column(String)
    type = Column(String)
    age = Column(Integer)

# Right now, this table only exists in python and not in the actual database

In [8]:
Base.metadata.tables

immutabledict({'pet': Table('pet', MetaData(bind=None), Column('id', Integer(), table=<pet>, primary_key=True, nullable=False), Column('name', String(), table=<pet>), Column('type', String(), table=<pet>), Column('age', Integer(), table=<pet>), schema=None)})

# Create our database engine

In [9]:
engine = create_engine('sqlite:///pets.sqlite')

# This is where we create our tables in the database

In [10]:
Base.metadata.create_all(engine)

# The ORM’s “handle” to the database is the Session.

In [11]:
from sqlalchemy.orm import Session
session = Session(engine)

# Create Data

## - Note that adding to the session does not update the table. It queues up those queries.

In [12]:
session.add(Pet(name='Justin Timbersnake', type='snek', age=2))
session.add(Pet(name='Pawtrick Stewart', type='good boy', age=10))
session.add(Pet(name='Godzilla', type='iguana', age=1))
session.add(Pet(name='Marshmallow', type='polar bear', age=4))

## - seesion.add_all( )

In [27]:
session.add_all([
    Pet(name='Simon', type='snek', age=2),
    Pet(name='Jennifer', type='snek', age=2)
])

## The data hasn't been added yet

In [13]:
engine.execute('select * from pet').fetchall()

[]

## - We can use the new attribute to see the queue of data ready to go into the database

In [14]:
session.new

IdentitySet([<__main__.Pet object at 0x7ff0c0451910>, <__main__.Pet object at 0x7ff0c0451bb0>, <__main__.Pet object at 0x7ff0c0451c10>, <__main__.Pet object at 0x7ff0c0451ca0>])

## - commit() flushes whatever remaining changes remain to the database, and commits the transaction.

In [15]:
session.commit()

## - Nothing new to add

In [16]:
session.new

IdentitySet([])

# Read Data
## - query the database

In [19]:
session.query(Pet.name, Pet.type, Pet.age).all()

[('Justin Timbersnake', 'snek', 2),
 ('Pawtrick Stewart', 'good boy', 10),
 ('Godzilla', 'iguana', 1),
 ('Marshmallow', 'polar bear', 4)]

# Update Data

## - Create a query and then run update on it
## - filter_by( ) method

In [20]:
pet = session.query(Pet).filter_by(name="Marshmallow").first()
pet.age += 1

## - For modifications, we can use the dirty attribute

In [21]:
session.dirty

IdentitySet([<__main__.Pet object at 0x7ff0c0451ca0>])

## - Commit Transaction

In [22]:
session.commit()

## - Session is up-to-date

In [23]:
session.dirty

IdentitySet([])

In [24]:
session.query(Pet.id, Pet.name, Pet.type, Pet.age).all()

[(1, 'Justin Timbersnake', 'snek', 2),
 (2, 'Pawtrick Stewart', 'good boy', 10),
 (3, 'Godzilla', 'iguana', 1),
 (4, 'Marshmallow', 'polar bear', 5)]

# Delete Data

## - Create a query and then delete the row collected
## - filter_by( )
## - one( )

In [18]:
pet = session.query(Pet).filter_by(id=4).one()

## - For delete, use delete( ) method

In [ ]:
session.delete(pet)

## - Use deleted attribute to check if the delete action has been to database or not
- The result show the delete action has not been executed iin database

In [19]:
session.deleted

IdentitySet([<__main__.Pet object at 0x7fd1b5b85f90>])

## - Commit the delete

In [20]:
session.commit()

## - Use deleted attribute to check
- The empty list shows the delete had been commited to database

In [21]:
session.deleted

IdentitySet([])

In [22]:
session.query(Pet.id, Pet.name, Pet.type, Pet.age).all()

[(1, 'Justin Timbersnake', 'snek', 2),
 (2, 'Pawtrick Stewart', 'good boy', 10),
 (3, 'Godzilla', 'iguana', 1)]

# Close the session

In [26]:
session.close()